In [1]:
import pandas as pd 
import numpy as np

In [ ]:
from ast import literal_eval

## Load the file with all scraped metadata (authors, titles, dates, abstracts...)
df = pd.read_csv("./FullScraped.csv", index_col=0)

## Get the abstracts - they will be used to generate question and labels
abstracts = df.abstract.tolist()


## Parse the abstracts into background, objectives, conclusions

backgrounds = list()
objectives = list()
conclusions = list()

missing_indices = list()
idx = 0
for ab in abstracts:
    ab = literal_eval(ab)
    
    bfound = False
    ofound = False
    cfound = False

    for elem_idx in range(len(ab)):
        elem = ab[elem_idx]
        if not bfound and "background:" in elem.lower():
            if elem.lower().strip().split("background:")[1].strip() != "":
                remainder = elem.lower().strip().split("background:")[1].strip()
                back = remainder.strip().split("objectives")[0].strip()
                backgrounds.append(back)
                bfound = True
                continue
            
            else:
                backgrounds.append(ab[elem_idx+1])
            bfound = True
            
        elif not ofound and "objectives:" in elem.lower():
            if elem.lower().strip().split("objectives:")[1].strip() != "":
                remainder = elem.lower().strip().split("objectives:")[1].strip()
                obj = remainder.strip().split("search methods")[0].strip()
                objectives.append(obj)
                ofound = True
                continue            
            elif elem_idx+1 == len(ab):
                objectives.append(elem.lower().split("objectives:")[1].strip())
                ofound = True
            else:
                objectives.append(ab[elem_idx+1])
                ofound = True
            
        elif not cfound and "conclusions:" in elem.lower():
            if elem.lower().strip().split("conclusions:")[1].strip() != "":
                remainder = elem.lower().strip().split("conclusions:")[1].strip()
                con = remainder
                conclusions.append(con)
                cfound = True
                continue
            elif elem_idx+1 == len(ab):
                conclusions.append(elem.lower().split("conclusions:")[1].strip())
            else:
                conclusions.append(ab[elem_idx+1])
            cfound = True
    if not bfound:
        backgrounds.append("None")
    if not ofound:
        objectives.append("None")
    if not cfound:
        conclusions.append("None")
    
    idx += 1



In [ ]:
## Questions and labels are generated based on study's background, objectives, conclusions
## Here we prepare the messages for the LLM

all_messages = list()
for idx in range(len(backgrounds)):

    system_message = "You're a helpful assistant. Your task is to help with generating questions and labels in the medical and clinical domain."
    user_message_begin = f'''
       You will be given an excerpt of an abstract of a clinical systematic review. Based on the given background, objectives, and author's conclusions, generate only ONE SINGLE question, answerable with yes/no/maybe, that sums up the main medical objective that was investigated. Please keep the question short and general, understandable to lay people. Please use the "OBJECTIVES" section as the basis to construct the question! Please do NOT mention the word review or studies anywhere in the question! The question should be about a general medical hypothesis, not about this specific review.
    
    Afterward, please also give a label for the author's conclusions. Label tries to answer the objective by looking at the conclusion. The label may be ONLY from one of the following three: (1) SUPPORTED; (2) REFUTED; (3) NOT ENOUGH INFORMATION. Do not try to make up a new label. Please only select the third label if not enough studies were found, not if certainty of the conclusion is low! Please strive to predict "REFUTED" or "SUPPORTED" even if certainty of these conclusions by authors is low or weak!  Label "SUPPORTED" means the objective's hypothesis is at least partially supported by the conclusion, label "REFUTED" means it is at least partially not supported or similar to placebo. 
    
    Afterward, please also give the level of evidence (strength of finding) in the author's conclusions. The label may be ONLY from one of the following three: (1) LOW; (2) MEDIUM; (3) HIGH . Do not try to make up a new label.
    
    Please structure the output in two lines, as:
    QUESTION: (question)
    LABEL: (label)
    LEVEL: (label)

    The documents begins now.
    BACKGROUND:
    '''

    finalstring = ""
    finalstring += user_message_begin
    finalstring += backgrounds[idx] + "\n"

    user_message_middle = f'''OBJECTIVES: 
    '''

    finalstring += user_message_middle
    finalstring += objectives[idx]  + "\n"

    user_message_end = f'''AUTHOR'S CONCLUSIONS:
    '''
    
    finalstring += user_message_end
    finalstring += conclusions[idx]

    messages =  [  
            {'role':'system', 
             'content': system_message},    
            {'role':'user', 
             'content': f"{finalstring}"},  
    ] 
    
    all_messages.append(messages)


In [ ]:
from openai import OpenAI

apikey = 'xyz'
client = OpenAI(api_key=apikey)

def query_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini", 
        max_tokens = 4096,
        messages=prompt, 
        temperature=0) 
    return response

## Generate questions and labels for all abstracts and save to file
with open("generated_questions_labels.txt", "w") as f:
    for msgs in all_messages:
        response = query_gpt(msgs)
        result = response.choices[0].message.content.strip()
        result = result.replace("\n", " ||| ")
        f.write(result)
        f.write("\n")
